# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from selenium import webdriver
# 選擇選單
from selenium.webdriver.support.ui import Select


In [2]:
# 擷取網址
url = "https://www.ettoday.net/news/news-list.htm"

r = requests.get(url)
soup = BeautifulSoup(r.text, "lxml")

## 1. 取出今天所有的新聞

In [3]:
#  設置日期
#  timedelta(1) 預設為1日
date_time = datetime.now() - timedelta(1)
date_time = date_time.strftime("%Y/%m/%d")
print("取得昨天時間：", date_time)

# 利用 selenium 持續執行下滑的動作，直到取得「昨天時間」的新聞，表示今天的已經取得完畢

browser = webdriver.Chrome(executable_path='chromedriver')
browser.get(url)

while True:
    # scrollTo 的設置是指 total scroll 的長度，而不是單次 scroll 的長度，這會影響擷取的次數
    browser.execute_script("window.scrollTo(0, 100000);")
    time.sleep(2)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "lxml")
    print(soup.find_all(class_="part_list_2")[-1].find_all("h3")[-1].find(class_="date").text)
    if date_time in soup.find_all(class_="part_list_2")[-1].find_all("h3")[-1].find(class_="date").text:
        print("===STOP===")
        break
        
html_source = browser.page_source
soup = BeautifulSoup(html_source, "lxml")

for d in soup.find(class_="part_list_2").find_all("h3"):
    if date_time in d.find(class_="date").text:
        break
    print(d.find(class_="date").text, d.find_all("a")[-1].text)

# 資料整理
news = []

for d in soup.find(class_="part_list_2").find_all("h3"):
    if date_time in d.find(class_="date").text:
        break
    news.append({
        '時間': d.find(class_="date").text,
        '標題': d.find_all("a")[-1].text
    })

# pd.DataFrame(news)

browser.close()

取得昨天時間： 2020/06/14
2020/06/15 09:47
2020/06/15 09:12
2020/06/15 09:00
2020/06/15 08:32
2020/06/15 07:56
2020/06/15 07:20
2020/06/15 06:03
2020/06/15 02:07
2020/06/15 00:13
2020/06/14 23:21
===STOP===
2020/06/15 11:45 光寶股東會通過：每股配息3.2元　現金殖利率6.3%！EPS年增18%「創3年來新高」
2020/06/15 11:44 陳其邁躺著選？　名嘴點出「兩大挑戰」：他應該緊張！高雄市民很累
2020/06/15 11:44 大甲媽祝壽大典改奉天宮廟內舉行　拜豬公求智慧毛只剩1頭
2020/06/15 11:44 改個錯字「網路誹謗罪」成立！　菲律賓《時代》年度風雲人物面臨7年監禁
2020/06/15 11:43 北京「4天62確診」開始究責　豐台區副區長、疫情爆發市場總經理全下台！
2020/06/15 11:41 iPhone 12系列生產模具曝光！間接說明 5 項新機傳聞
2020/06/15 11:41 找室內設計師來估價…卻要先付「5000元出圖費」！她傻了：業界都這樣？
2020/06/15 11:39 把家寫成「厝」　閩南語漢字專家：自己在詛咒自己
2020/06/15 11:37 黃捷回嗆：鳳山的「鳳」不是那麼好寫　喊「罷免要先想好理由」
2020/06/15 11:37 雞排妹「3個月菜鳥」股票爽撈50萬！曾被交易員勸退...她堅持：26歲人生解鎖
2020/06/15 11:35 「砸出球棒印」！路怒族汐止塞車火氣大　超車不成怒砸前車後擋風玻璃
2020/06/15 11:35 「這條街最醜房」！夫妻倆買下「灌入時尚元素」　超美落地窗秒成當地網紅景點
2020/06/15 11:34 沈榮津接行政院副院長？　陳其邁大讚：史上最強歐吉桑
2020/06/15 11:33 悠遊付祭優惠　共享機車會員1280定期票打7折
2020/06/15 11:33 北京疫情傳播鏈已跨省！河北一家四人染疫「6歲女童去過幼稚園」
2020/06/15 11:32 單驥／廣明光電滅頂賠償案：科技戰後的法律戰
2020/06/15 11:31 鳳凰

## 2. 取出現在時間兩小時內的新聞

In [4]:
two_hour_date_time = datetime.now() - timedelta(hours = 2)

for d in soup.find(class_="part_list_2").find_all("h3"):
    if datetime.strptime(d.find(class_="date").text, "%Y/%m/%d %H:%M") < two_hour_date_time:
        break
    news.append({
        '時間': d.find(class_="date").text,
        '標題': d.find_all("a")[-1].text
    })
pd.DataFrame(news)


,時間,標題
0,2020/06/15 11:45,光寶股東會通過：每股配息3.2元 現金殖利率6.3%！EPS年增18%「創3年來新高」
1,2020/06/15 11:44,陳其邁躺著選？ 名嘴點出「兩大挑戰」：他應該緊張！高雄市民很累
2,2020/06/15 11:44,大甲媽祝壽大典改奉天宮廟內舉行 拜豬公求智慧毛只剩1頭
3,2020/06/15 11:44,改個錯字「網路誹謗罪」成立！ 菲律賓《時代》年度風雲人物面臨7年監禁
4,2020/06/15 11:43,北京「4天62確診」開始究責 豐台區副區長、疫情爆發市場總經理全下台！
...,...,...
298,2020/06/15 09:54,大甲媽祖出巡牽紅線 超市「紅利5倍送」再抽百枚媽祖過火加持金牌
299,2020/06/15 09:53,果嶺最性感選手示範「巨乳夾桿」 拍裸照反網路霸凌
300,2020/06/15 09:50,尿有泡泡不等於是「腎虧」！ 是不是腎臟病要看泡泡持續多久！
301,2020/06/15 09:48,蘆洲「首購天堂」出列！當地人稱最強商圈捷運宅 入手價700萬起


## 3. 根據範例，取出三天前下午三點到五點的新聞

In [5]:
browser = webdriver.Chrome(executable_path='chromedriver')

browser.get("https://www.ettoday.net/news/news-list.htm")

three_days_date_time = datetime.now() - timedelta(3)
print('取得三天前的時間：', three_days_date_time.strftime("%Y/%m/%d"))

three_days_month = three_days_date_time.strftime("%-m")
three_days_day = three_days_date_time.strftime("%-d")

print('取得三天前的月：',three_days_month)
print('取得三天前的日：',three_days_day)

# strptime () 函數根據指定的格式把一個時間字符串解析為時間元組。
# strftime () 函數接收以時間元組，並返回以可讀字符串表示的當地時間，格式由參數 format 決定。

three_days_15_clock = datetime.strptime(three_days_date_time.strftime("%Y/%m/%d") + " 15", '%Y/%m/%d %H')
three_days_17_clock = datetime.strptime(three_days_date_time.strftime("%Y/%m/%d") + " 17", '%Y/%m/%d %H')

print('取得三天前下午三點：',three_days_15_clock)
print('取得三天前下午五點：',three_days_17_clock)

取得三天前的時間： 2020/06/12
取得三天前的月： 6
取得三天前的日： 12
取得三天前下午三點： 2020-06-12 15:00:00
取得三天前下午五點： 2020-06-12 17:00:00


In [6]:
selM = Select(browser.find_element_by_id("selM"))
selM.select_by_value(three_days_month)
selD = Select(browser.find_element_by_id("selD"))
selD.select_by_value(three_days_day)
browser.find_element_by_id("button").click()

In [7]:
while True:
    browser.execute_script("window.scrollTo(0, 100000);")
    time.sleep(2)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "lxml")
    new_date = soup.find_all(class_="part_list_2")[-1].find_all("h3")[-1].find(class_="date").text
    print(new_date)
    if datetime.strptime(new_date, "%Y/%m/%d %H:%M") < three_days_15_clock:
        print('====STOP====')
        break


2020/06/12 19:39
2020/06/12 19:25
2020/06/12 19:05
2020/06/12 18:58
2020/06/12 18:51
2020/06/12 18:45
2020/06/12 18:32
2020/06/12 18:23
2020/06/12 18:17
2020/06/12 18:06
2020/06/12 17:57
2020/06/12 17:50
2020/06/12 17:44
2020/06/12 17:36
2020/06/12 17:30
2020/06/12 17:18
2020/06/12 17:07
2020/06/12 17:01
2020/06/12 16:55
2020/06/12 16:37
2020/06/12 16:25
2020/06/12 16:20
2020/06/12 16:10
2020/06/12 16:05
2020/06/12 15:56
2020/06/12 15:48
2020/06/12 15:40
2020/06/12 15:31
2020/06/12 15:25
2020/06/12 15:15
2020/06/12 15:07
2020/06/12 15:01
2020/06/12 14:56
====STOP====


In [8]:
news = []
for d in soup.find(class_="part_list_2").find_all("h3"):
    # 比兩小時前早的話就結束
    if datetime.strptime(d.find(class_="date").text, "%Y/%m/%d %H:%M") < three_days_15_clock:
        print("===stop===")
        break
    if datetime.strptime(d.find(class_="date").text, "%Y/%m/%d %H:%M") < three_days_17_clock:
        news.append({
            '時間': d.find(class_="date").text,
            '標題': d.find_all("a")[-1].text
        })
        
pd.DataFrame(news)

===stop===


,時間,標題
0,2020/06/12 16:58,台南北門釣客「踩破水泥橋」落水 民眾急拋繩力抗急流救一命
1,2020/06/12 16:57,蘭嶼出現Uber Eats外送機車 鄉民大樂「終於有了！」答案讓人跌破眼鏡
2,2020/06/12 16:57,尋找你的甲蟲王 《小森生活》全新甲蟲對戰系統上線
3,2020/06/12 16:56,「Galaxy Tab S7」渲染圖流出！ 傳機身側面新增指紋感應
4,2020/06/12 16:55,印尼看護才剛繳清仲介費卻罹罕病！台人暖募40萬...助母妹來台捐骨髓救命
...,...,...
134,2020/06/12 15:01,「小突兔主題餐廳與限定店」快閃三創 超可愛奶酪＋杯墊手刀衝一波
135,2020/06/12 15:01,《最後生還者二部曲》上市前無雷評測 跟著艾莉踏上「胃絞痛」的復仇之旅
136,2020/06/12 15:00,快訊／台南警「追落跑嫌疑犯」！路口撞上機車 25歲騎士重傷搶救中
137,2020/06/12 15:00,川普讚國民兵鎮暴是美麗風景線 衝進人群像刀切奶油！


In [9]:
browser.close()